In [18]:
import os
os.chdir("C:/Users/RAKESH MOURYA/Desktop/Bike_Sharing_Prediction/Bike_Sharing_Prediction")
%pwd

'C:\\Users\\RAKESH MOURYA\\Desktop\\Bike_Sharing_Prediction\\Bike_Sharing_Prediction'

In [19]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class datapreprocessing:
    root_dir: Path
    n_features_to_select: int
    data_path: Path
    transformed_train_path: Path
    transformed_test_path: Path
    preprocessor_obj_path: Path
    test_size: float = 0.3
    random_state: int = 100

In [20]:
from bike_rental.constants import*
from bike_rental.utils.common import read_yaml, create_directories

In [22]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
        
        
    def get_data_preprocessing(self) ->datapreprocessing:
        
        config=self.config.data_preprocessing
        create_directories([config.root_dir])
        
        data_preprocessing_config =datapreprocessing(
            root_dir=Path(config.root_dir),
            n_features_to_select=self.params.n_features_to_select,
            data_path=Path(config.data_path),
            transformed_train_path=Path(config.transformed_train_path),
            transformed_test_path=Path(config.transformed_test_path),
            preprocessor_obj_path=Path(config.preprocessor_obj_path)      
        )
        
        return data_preprocessing_config
       

In [23]:
import os
import urllib.request as request
from bike_rental import logger
from bike_rental.utils.common import get_size

In [24]:
import os
import urllib.request as request
from zipfile import ZipFile
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns 
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import statsmodels.api as sm
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression 
from statsmodels.stats.outliers_influence import variance_inflation_factor


In [30]:
from bike_rental import logger
import pickle
class data_preprocessing:
    def __init__(self,config: datapreprocessing):
        self.config=config
    
    def transform_data(self):
        logger.info("Reading dataset for preprocessing...")
        df = pd.read_csv(self.config.data_path)

        # --- your mapping + dummy code ---
        df['season'] = df.season.map({1:'spring',2:'summer',3:'fall',4:'winter'})
        df['mnth'] = df.mnth.map({1:'Jan',2:'Feb',3:'Mar',4:'Apr',5:'May',6:'June',
                                  7:'July',8:'Aug',9:'Sep',10:'Oct',11:'Nov',12:'Dec'})
        df['weathersit'] = df.weathersit.map({1:'Clear',2:'Mist + Cloudy',3:'Light Snow',4:'Snow + Fog'})
        df['weekday'] = df.weekday.map({0:'Sun',1:'Mon',2:'Tue',3:'Wed',4:'Thu',5:'Fri',6:'Sat'})

        df = df.drop(['atemp','instant','dteday','casual','registered'], axis=1)

        month = pd.get_dummies(df.mnth, drop_first=True)
        weekday = pd.get_dummies(df.weekday, drop_first=True)
        weathersit = pd.get_dummies(df.weathersit, drop_first=True)
        season = pd.get_dummies(df.season, drop_first=True)

        df = pd.concat([df, month, weekday, weathersit, season], axis=1)
        df.drop(['season','mnth','weekday','weathersit'], axis=1, inplace=True)
        
        scaler = MinMaxScaler()
        scaler_var = ['hum', 'windspeed', 'temp', 'cnt']
        df[scaler_var] = scaler.fit_transform(df[scaler_var])
        
        # --- Split ---
        train, test = train_test_split(df, test_size=0.3, random_state=100)
        train.to_csv(self.config.transformed_train_path, index=False)
        test.to_csv(self.config.transformed_test_path, index=False)

        # --- Save preprocessor object (optional placeholder now) ---
        with open(self.config.preprocessor_obj_path, "wb") as f:
            pickle.dump({"mapping": "done"}, f)

        logger.info(f"Preprocessing complete. Train & test saved at {self.config.root_dir}")

In [31]:
try:
    config = ConfigurationManager()
    data_preprocessing_config = config.get_data_preprocessing()
    data_transforming = data_preprocessing(config=data_preprocessing_config)
    data_transforming.transform_data()
except Exception as e:
    raise e

[2025-09-13 16:05:09,110: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-09-13 16:05:09,118: INFO: common: yaml file: params.yaml loaded successfully]
[2025-09-13 16:05:09,126: INFO: common: created directory at: artifacts]
[2025-09-13 16:05:09,134: INFO: common: created directory at: artifacts/data_preprocessing]
[2025-09-13 16:05:09,134: INFO: 1727429553: Reading dataset for preprocessing...]


[2025-09-13 16:05:09,310: INFO: 1727429553: Preprocessing complete. Train & test saved at artifacts\data_preprocessing]
